In [1]:
from Bubble_dynamics_simulation import inp_data_extractor as inp

path = 'Bubble_dynamics_simulation\\INP file examples\\chemkin_AR_HE.inp'
inp.extract(path)
import parameters as par

path=Bubble_dynamics_simulation\INP file examples\chemkin_AR_HE.inp
Note, lambda value for specie 'H' is not in data.py: 0.0 is used
Note, lambda value for specie 'O' is not in data.py: 0.0 is used
Note, lambda value for specie 'OH' is not in data.py: 0.0 is used
Note, lambda value for specie 'HO2' is not in data.py: 0.0 is used
Note, lambda value for specie 'OHEX' is not in data.py: 0.0 is used
Warning, third body 'CO' is not in species in line 65 ('H2/2.5/ O2/1/ H2O/12/ CO/1.9/ CO2/3.8/ HE/0.83/ AR/1/') in reaction '2H+M=H2+M'
Warning, third body 'CO2' is not in species in line 65 ('H2/2.5/ O2/1/ H2O/12/ CO/1.9/ CO2/3.8/ HE/0.83/ AR/1/') in reaction '2H+M=H2+M'
Warning, third body 'CO' is not in species in line 67 ('H2/2.5/ H2O/12/ AR/0.83/ CO/1.9/ CO2/3.8/ HE/0.83/') in reaction '2O+M=O2+M'
Warning, third body 'CO2' is not in species in line 67 ('H2/2.5/ H2O/12/ AR/0.83/ CO/1.9/ CO2/3.8/ HE/0.83/') in reaction '2O+M=O2+M'
Warning, third body 'CO' is not in species in line 69 ('H2/2.

In [2]:
# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

import numpy as np   # matrices, math
import time   # runtime measurement
from multiprocessing import Pool, cpu_count   # multithreading
import importlib   # reload changes you made
import itertools   # assemble all combinations of control 
import json   # convert dictionary to string
from termcolor import colored   # colored text in terminal

# my own file:
import Bubble_dynamics_simulation.full_bubble_model as de
import Bubble_dynamics_simulation.full_bubble_model_old as deo

model: chemkin_AR_HE
target specie: H2
excitation: sin_impulse (control parameters: ['p_A', 'freq', 'n'])
enable heat transfer: True	enable evaporation: True	enable reactions: True	enable dissipated energy: True
model: chemkin_AR_HE
target specie: H2
excitation: sin_impulse (control parameters: ['p_A', 'freq', 'n'])
enable heat transfer: True	enable evaporation: True	enable reactions: True	enable dissipated energy: True


In [3]:
cpar = de.dotdict(dict(
    ID = 0,                                      # ID of control parameter (not used during calculation)
  # Initial conditions:
    R_E =  0.00001000,                           # bubble equilibrium radius [m]
    ratio =  1.00,                               # initial radius / equilibrium radius R_0/R_E [-]
    gases = [par.index['AR']],  # indexes of species in initial bubble (list of species indexes)
    fractions = [1.0],                    # molar fractions of species in initial bubble (list of fractions for every gas)
  # Ambient parameters:
    P_amb =  101325.00,                          # ambient pressure [Pa]
    T_inf =  293.15,                             # ambient temperature [K]
  # Liquid parameters:
    alfa_M =  0.3500,                            # water accommodation coefficient [-]
    P_v =  2338.10,                              # vapour pressure [Pa]
    mu_L =  0.0010,                              # dynamic viscosity [Pa*s]
    c_L =  1483.00,                              # sound speed [m/s]
    surfactant =  1.00,                          # surfactant (surface tension modfier) [-]
  # Excitation parameters: (excitation_type = sin_impulse)
    p_A = -3e5,                                  # [Pa]
    freq = 20000.0,                              # [Hz]
    n = 1.0,                                     # [-]
))

# Calculate pressure/temperature dependent parameters:
cpar.mu_L = de.Viscosity(cpar.T_inf)
cpar.P_v = de.VapourPressure(cpar.T_inf)

In [4]:
IC, _, _ = de.InitialCondition(cpar)
kwargs = dict(t=0.0, x=IC, P_amb=1e5, alfa_M=0.35, T_inf=300.0, surfactant=1.0, P_v=par.P_v, mu_L=par.mu_L, c_L=par.c_L, ex_args=np.array([-3e5, 20000.0, 1.0], dtype=np.float64))
%timeit de.f(**kwargs)
%timeit deo.f(**kwargs)

5.85 µs ± 62.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
17 µs ± 149 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
def equal(eps=1e-20, **kwargs):
    name1, name2 = kwargs.keys()
    a = kwargs[name1]
    b = kwargs[name2]
    if type(a) == tuple or type(a) == list:
        a = np.array(a)
        b = np.array(b)
    if type(a) == np.ndarray:
        if type(a[0]) == np.ndarray:
            ret = True
            for i, (aa, bb) in enumerate(zip(a, b)):
                ret = ret and equal(eps, **{name1+str(i): aa, name2+str(i): bb})
            return ret
        else:
            diff = np.abs(a - b) / (abs(b) + 1e-30)
            wrong = np.where(diff > eps)[0]
            if len(wrong) > 0:
                print(colored(f'{name1} != {name2} at indexes:', 'red'), wrong)
                for i in wrong:
                    print(f'\t{i}: {a[i]: e} != {b[i]: e}\t (diff: {diff[i]: e})')
                return False
            return True
    else:
        diff = abs(a - b) / (abs(b) + 1e-30)
        if diff > eps:
            print(colored(f'{name1} != {name2}', 'red'))
            print(f'\t{a: e} != {b: e}\t (diff: {diff: e})')
            return False
        return True
    
def equal_data(a, b, eps=1e-20):
    ret = True
    ret = ret and equal(eps, x_initial1=a.x_initial, x_initial2=b.x_initial)
    ret = ret and equal(eps, x_final1=a.x_final, x_final2=b.x_final)
    ret = ret and equal(eps, n_NH31=a.n_H2, n_NH32=b.n_H2)
    ret = ret and equal(eps, dissipated_acoustic_energy1=a.dissipated_acoustic_energy, dissipated_acoustic_energy2=b.dissipated_acoustic_energy)
    ret = ret and equal(eps, energy_efficiency1=a.energy_efficiency, energy_efficiency2=b.energy_efficiency)
    ret = ret and equal(eps, T_max1=a.T_max, T_max2=b.T_max)
    return ret


In [9]:
new = de.simulate(dict(cpar=cpar))
old = deo.simulate(dict(cpar=cpar))
equal_data(de.dotdict(new), de.dotdict(old))

C:\Users\aron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ode.py:1348: UserWarning: lsoda: Excess accuracy requested (tolerances too small).
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


True

In [10]:
num_sol, _, _ = de.solve(cpar)
for i in range(0, len(num_sol.t), 10):
    t = num_sol.t[i]
    x = num_sol.y[:, i]
    kwargs = dict(t=t, x=x, P_amb=cpar.P_amb, alfa_M=cpar.alfa_M, T_inf=cpar.T_inf, surfactant=cpar.surfactant,
                  P_v=cpar.P_v, mu_L=cpar.mu_L, c_L=cpar.c_L, ex_args=np.array([cpar[key] for key in de.excitation_args], dtype=np.float64))
    new = de.f(**kwargs)
    old = deo.f(**kwargs)
    if not equal(new=new, old=old, eps=1e-100):
        print(i)
        break

In [13]:
num_sol1, error1, elapsed1 = de.solve(cpar, LSODA_timeout=100, Radau_timeout=100)
num_sol2, error2, elapsed2 = deo.solve(cpar, LSODA_timeout=100, Radau_timeout=100)
print('\tnew', '\told')
print('elapsed', elapsed1, elapsed2)
print('error', error1, error2)
print('shape', num_sol1.y.shape, num_sol2.y.shape)

	new 	old
elapsed 2.240536689758301 3.4287993907928467
error 1 1
shape (16, 8371) (16, 8371)


In [16]:
kwargs = dict(t=t, x=x, P_amb=cpar.P_amb, alfa_M=cpar.alfa_M, T_inf=cpar.T_inf, surfactant=cpar.surfactant, P_v=cpar.P_v, mu_L=cpar.mu_L, c_L=cpar.c_L, ex_args=np.array([cpar[key] for key in de.excitation_args], dtype=np.float64))
print(colored('Pressure', 'green'))
R = x[0]      # bubble radius [m]
R_dot = x[1]  # [m/s]
T = x[2]      # temperature [K]
c = x[3:-1]     # molar concentration [mol/cm^3]
M = np.sum(c) # sum of concentration
X = c / M     # mole fraction [-]
p = 0.1 * M * par.R_erg * T # Partial pressure of the gases [Pa]
new = de.Pressure(t, R, R_dot, cpar.mu_L, cpar.surfactant, p, 3.14, cpar.P_amb, np.array([cpar.p_A, cpar.freq, cpar.n], dtype=np.float64))
old = deo.Pressure(t, R, R_dot, cpar.mu_L, cpar.surfactant, p, 3.14, cpar.P_amb, np.array([cpar.p_A, cpar.freq, cpar.n], dtype=np.float64))
equal(new=new, old=old, eps=1e-100)

print(colored('Thermodynamic', 'green'))
new = de.Thermodynamic(T)
old = deo.Thermodynamic(c, T)
equal(new=new, old=old, eps=1e-100)
[C_p, H, S, C_v] = new

if de.enable_evaporation:
    print(colored('Evaporation', 'green'))
    X_H2O = X[par.index['H2O']]
    new = de.Evaporation(p, T, X_H2O, cpar.alfa_M, cpar.T_inf, cpar.P_v)
    old = deo.Evaporation(M, T, X_H2O, cpar.alfa_M, cpar.T_inf, cpar.P_v)
    equal(new=new, old=old, eps=1e-100)

print(colored('Forward rate', 'green'))
M_eff1 = np.zeros((par.ThirdBodyCount), dtype = np.float64)   # effective total concentration of the third-body 
for j, i in enumerate(par.ThirdBodyIndexes):
    M_eff1[j] = np.sum(par.alfa[j] * c)
M_eff2 = np.sum(c) * np.ones((par.I), dtype = np.float64)    # effective total concentration of the third-body 
for j, i in enumerate(par.ThirdBodyIndexes):
    M_eff2[i] = np.sum(par.alfa[j] * c)
new = de.ForwardRate(T, M_eff1, M, p)
old = deo.ForwardRate(T, M_eff2, M)
equal(new=new, old=old, eps=1e-100)
k_forward = new

print(colored('Backward rate', 'green'))
new= de.BackwardRate(k_forward, S, H, T, cpar.P_amb)
old= deo.BackwardRate(k_forward, S, H, T, cpar.P_amb)
equal(new=new, old=old, eps=1e-100)

print(colored('Production rate', 'green'))
new= de.ProductionRate(T, H, S, c, cpar.P_amb, p, M)
old= deo.ProductionRate(T, H, S, c, cpar.P_amb, M)
equal(new=new, old=old, eps=1e-100)

print(colored('ODE fun', 'green'))
new = de.f(t, x, cpar.P_amb, cpar.alfa_M, cpar.T_inf, cpar.surfactant, cpar.P_v, cpar.mu_L, cpar.c_L, np.array([cpar.p_A, cpar.freq, cpar.n], dtype=np.float64))
old = deo.f(t, x, cpar.P_amb, cpar.alfa_M, cpar.T_inf, cpar.surfactant, cpar.P_v, cpar.mu_L, cpar.c_L, np.array([cpar.p_A, cpar.freq, cpar.n], dtype=np.float64))
equal(new=new, old=old, eps=1e-100)
None

Pressure
Thermodynamic
Evaporation
Forward rate
Backward rate
Production rate
ODE fun


In [27]:
ranges = dict(
  # Initial conditions:
    # bubble equilibrium radius [m]
    R_E = [1e-6*x for x in [5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 50.0, 60.0]], # [um --> m]
    # initial radius / equilibrium radius R_0/R_E [-]
    ratio = [1.0],
  # Ambient parameters:
    # ambient pressure [Pa]
    P_amb = [x * par.atm2Pa for x in [0.5, 0.75, 1.0, 1.25, 1.5]], # [atm --> Pa]
    # ambient temperature [K]       
    T_inf = [par.absolute_zero + x for x in [30.0]], # [°C --> K]
  # Liquid parameters:
    # water accommodation coefficient [-]
    alfa_M = [0.35],
    #P_v = par.P_v,                    # vapour pressure [Pa]
    #mu_L = par.mu_L,                  # dynamic viscosity [Pa*s]
    # sound speed [m/s]
    c_L = [par.c_L],
    # surfactant (surface tension modfier) [-]
    surfactant = [1.0],
  # Excitation parameters: (excitation_type = no_excitation)
    p_A = [-1e5*x for x in [2.0, 2.5, 3.0, 3.5, 4.0]], # [Pa]
    freq = [20000.0, 40000.0], # [Hz]
    n = [1.0], # [-]
)

cpars = []
ID = 1
for values in itertools.product(*ranges.values()):
    cpar = dict(zip(ranges.keys(), values))
    cpar['ID'] = ID                      # ID of control parameter (not used during calculation)
    cpar['gases'] = [par.index['AR']]    # indexes of species in initial bubble (list of species indexes)
    cpar['fractions'] = [1.0]            # molar fractions of species in initial bubble (list of fractions for every gas)
    # Calculate pressure/temperature dependent parameters:
    cpar['mu_L'] = de.Viscosity(cpar['T_inf'])
    cpar['P_v'] = de.VapourPressure(cpar['T_inf'])
    cpars.append(cpar)
    ID += 1

# Create input dictionary for de.simulate(), a list of dictionaries with cpar and other arguments
kwargs_list = [dict(cpar=cpar, t_int=np.array([0.0, 1.0]), LSODA_timeout=100, Radau_timeout=0) for cpar in cpars]


best_energy_efficiency = 1e30
start = time.time()
with Pool(processes=cpu_count(), maxtasksperchild=100) as pool:
    results = pool.imap_unordered(de.simulate, kwargs_list)

    for data in results:
      # save results:
        data = de.dotdict(data)
      # print stuff:
        if data.energy_efficiency > 0 and data.energy_efficiency < best_energy_efficiency:
            best_energy_efficiency = data.energy_efficiency
        excitation_params = ''.join([f'{key}={data[key]: <12}; ' for key in de.excitation_args])
        print(f'index: {data.ID: >8}/{len(cpars)};   error_code: {data.error_code: >4};   steps: {data.steps: <8};   runtime: {data.elapsed_time: 6.2f} [s]   |   '+
              f'R_E={1e6*data.R_E: 6.2f} [um]; ratio={data.ratio: 6.2f} [-]; P_amb={1e-5*data.P_amb: 6.2f} [bar]; alfa_M={data.alfa_M: 6.2f} [-]; '+
              f'T_inf={data.T_inf-273.15: 6.2f} [°C]; surfactant={100*data.surfactant: 3.0f} [%]   |   {excitation_params}   |   '+
              f'{de.target_specie} production: {data.energy_efficiency: e} [MJ/kg] (best: {best_energy_efficiency: .1f} [MJ/kg])'+
              '                                                 ', end='\r')
              
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f'total time: {(elapsed / 3600): .0f} hours {((elapsed % 3600) / 60): .0f} mins')
print(f'            {elapsed: .2f} [s]   ({(elapsed / len(    cpars)): .2f} [s/run])')

In [ ]:
best_energy_efficiency = 1e30
start = time.time()
with Pool(processes=cpu_count(), maxtasksperchild=100) as pool:
    results = pool.imap_unordered(deo.simulate, kwargs_list)

    for data in results:
      # save results:
        data = de.dotdict(data)
      # print stuff:
        if data.energy_efficiency > 0 and data.energy_efficiency < best_energy_efficiency:
            best_energy_efficiency = data.energy_efficiency
        excitation_params = ''.join([f'{key}={data[key]: <12}; ' for key in de.excitation_args])
        print(f'index: {data.ID: >8}/{len(cpars)};   error_code: {data.error_code: >4};   steps: {data.steps: <8};   runtime: {data.elapsed_time: 6.2f} [s]   |   '+
              f'R_E={1e6*data.R_E: 6.2f} [um]; ratio={data.ratio: 6.2f} [-]; P_amb={1e-5*data.P_amb: 6.2f} [bar]; alfa_M={data.alfa_M: 6.2f} [-]; '+
              f'T_inf={data.T_inf-273.15: 6.2f} [°C]; surfactant={100*data.surfactant: 3.0f} [%]   |   {excitation_params}   |   '+
              f'{de.target_specie} production: {data.energy_efficiency: e} [MJ/kg] (best: {best_energy_efficiency: .1f} [MJ/kg])'+
              '                                                 ', end='\r')
              
end = time.time()
elapsed = end - start
print(f'\n\nDONE')
print(f'total time: {(elapsed / 3600): .0f} hours {((elapsed % 3600) / 60): .0f} mins')
print(f'            {elapsed: .2f} [s]   ({(elapsed / len(    cpars)): .2f} [s/run])')

index:      306/320;   error_code:    0;   steps: 220598  ;   runtime:  35.48 [s]   |   R_E= 60.00 [um]; ratio=  1.00 [-]; P_amb=  2.03 [bar]; alfa_M=  0.35 [-]; T_inf= 30.00 [°C]; surfactant= 100 [%]   |   p_A=-150000.0   ; freq=40000.0     ; n=1.0         ;    |   H2 production:  4.044754e+11 [MJ/kg] (best:  19903.9 [MJ/kg])                                                  

DONE
total time:  0 hours  2 mins
             94.82 [s]   ( 0.30 [s/run])


In [ ]:
new_time = 0.0
old_time = 0.0
for i, kwargs in enumerate(kwargs_list):
    start = time.time()
    new_data = de.simulate(kwargs)
    end = time.time()
    new_time += end - start
    start = time.time()
    old_data = deo.simulate(kwargs)
    end = time.time()
    old_time += end - start
    a = de.dotdict(new_data)
    b = de.dotdict(old_data)
    if a.error_code == b.error_code:
        ret = True
        eps = 1e-20
        ret = ret and equal(eps, x_initial1=a.x_initial, x_initial2=b.x_initial)
        ret = ret and equal(eps, x_final1=a.x_final, x_final2=b.x_final)
        ret = ret and equal(eps, n_NH31=a.n_NH3, n_NH32=b.n_NH3)
        ret = ret and equal(eps, dissipated_acoustic_energy1=a.dissipated_acoustic_energy, dissipated_acoustic_energy2=b.dissipated_acoustic_energy)
        ret = ret and equal(eps, energy_efficiency1=a.energy_efficiency, energy_efficiency2=b.energy_efficiency)
        ret = ret and equal(eps, T_max1=a.T_max, T_max2=b.T_max)
        if not ret: print(colored(f'ERROR: {i}', 'red'))
    else:
        print(colored(f'ERROR CODE MISMATCH: {i} ({a.error_code} != {b.error_code})', 'red'))
    print(f'{i}: {new_time=:.2f};    {old_time=:.2f};    {new_time/old_time=:.2f}                                    ', end='\r')

C:\Users\aron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ode.py:1348: UserWarning: lsoda: Excess accuracy requested (tolerances too small).
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


ERROR CODE MISMATCH: 18 (253 != 251)


C:\Users\aron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ode.py:1348: UserWarning: lsoda: Repeated convergence failures (perhaps bad Jacobian or tolerances).
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


ERROR CODE MISMATCH: 58 (253 != 251)_time/old_time=0.51                                    
ERROR CODE MISMATCH: 64 (253 != 251)_time/old_time=0.51                                    
ERROR CODE MISMATCH: 146 (253 != 251)ew_time/old_time=0.47                                    
ERROR CODE MISMATCH: 197 (253 != 251)ew_time/old_time=0.44                                    
ERROR CODE MISMATCH: 199 (200 != 252)ew_time/old_time=0.44                                    
ERROR CODE MISMATCH: 213 (253 != 251)new_time/old_time=0.49                                    
ERROR CODE MISMATCH: 236 (200 != 252)new_time/old_time=0.52                                    
ERROR CODE MISMATCH: 238 (200 != 252)new_time/old_time=0.51                                    
ERROR CODE MISMATCH: 239 (200 != 252)new_time/old_time=0.51                                    
ERROR CODE MISMATCH: 273 (200 != 252)new_time/old_time=0.51                                    
ERROR CODE MISMATCH: 276 (200 != 252) new_time/old_

In [ ]:
print(f'{new_time=:.2f};    {old_time=:.2f};    {new_time/old_time=:.2f}')
print(f'{new_time/len(kwargs_list):.2f};    {old_time/len(kwargs_list)=:.2f}')

new_time=2955.08;    old_time=5647.31;    new_time/old_time=0.52
7.39;    old_time/len(kwargs_list)=14.12


In [ ]:
i = 232
kwargs = kwargs_list[i]
new_data = de.simulate(kwargs)
old_data = deo.simulate(kwargs)

eps = 1e-20
a = de.dotdict(new_data)
b = de.dotdict(old_data)
equal(eps, steps1=a.steps, steps2=b.steps)
equal(eps, x_initial1=a.x_initial, x_initial2=b.x_initial)
equal(eps, x_final1=a.x_final, x_final2=b.x_final)
equal(eps, n_NH31=a.n_NH3, n_NH32=b.n_NH3)
equal(eps, dissipated_acoustic_energy1=a.dissipated_acoustic_energy, dissipated_acoustic_energy2=b.dissipated_acoustic_energy)
equal(eps, energy_efficiency1=a.energy_efficiency, energy_efficiency2=b.energy_efficiency)
equal(eps, T_max1=a.T_max, T_max2=b.T_max)


True

In [ ]:
"""
WindowsPC: Windows 11; AMD Ryzen 7 5800H (8*2 cores 3.2-4.4 GHz); 32 GB DDR4 RAM; Python 3.10.11;
LinuxPC: Ubuntu 22; Intel Core i7-3770 CPU (8 cores 3.4-3.9 GHz); 16 GB DDR3 RAM; Python 3.10.12

Ammonia: chem_Otomo2018_without_O; 12 species; 35 reactions; no_excitation; no evaporation;
Hydrogen: ???

There is always 100 s LSODA timeout and 0 s Radau timeout. Obviously the slower (old) code results in more timeouts.


Ammonia + WindowsPC: (400 combinations)
               | single-threaded | multi-threaded | ratio
---------------+-----------------+----------------+------
    new        |  2955.08 [s]    |  374.21 [s]    | 7.90
    old        |  5647.31 [s]    |  551.93 [s]    | 10.23
    ratio      |  0.52           |  0.67          |

Ammonia + LinuxPC: (400 combinations)
               | single-threaded | multi-threaded | ratio
---------------+-----------------+----------------+------
    new        | 7611.26  [s]    |  1456.92 [s]   | 5.23   
    old        | 10790.03 [s]    |  1736.31 [s]   | 6.21    
    ratio      | 0.70            |  0.84          |
"""